In [ ]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [ ]:
data = pd.read_csv('data/gb_sem_9_cw.csv')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
from operator import mul

def convert_time(my_time: str):
    factors = (1, 1/60)
    time = sum(map(mul, map(float, my_time.split(':')), factors))
    return round(time, 2)

data.time = data.time.apply(convert_time)
data.con_treat.replace({'control': 0, 'treatment': 1}, inplace=True)
data.page.replace({'old_page': 0, 'new_page': 1}, inplace=True)

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.con_treat.equals(data.page)

In [ ]:
data.con_treat.compare(data.page).index

In [ ]:
data.iloc[data.con_treat.compare(data.page).index, :]

In [ ]:
data_2 = data.drop(data.con_treat.compare(data.page).index).copy(deep=True)

In [ ]:
data_2.con_treat.equals(data_2.page)

In [ ]:
data_2.id.value_counts()

In [ ]:
data_2 = data_2.loc[data_2.id.isin(data_2.id.value_counts()[data_2.id.value_counts() == 1].index.values), :]

In [ ]:
data_2

In [ ]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [ ]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [ ]:
control = data_2[data_2.con_treat == 0].copy(deep=True)
treatment = data_2[data_2.con_treat == 1].copy(deep=True)

In [ ]:
### Testing timespent
control.shape, treatment.shape

In [ ]:
fig = px.histogram(data,
                   x='time',
                   color = 'con_treat',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

In [ ]:
continious_result(control, treatment, 'time')

In [ ]:
### Bucket

In [ ]:
for _ in range(100, 1001): 
    if data_2.shape[0] % _ == 0:
        print(_)

In [ ]:
n_buckets = 747
data_2 = (data_2
 .sample(n=data_2.shape[0], replace=False)
 .reset_index(drop=True)
 .assign(bucket=list(range(n_buckets)) * int(data_2.shape[0] / n_buckets)))

In [ ]:
data_2.head()

In [ ]:
bucketed_data_2 = data_2.groupby(['con_treat', 'bucket'])['time'].agg(mu=np.mean, std=np.std).reset_index()
bucketed_data_2

In [ ]:
# Сравним исходное выборочное среднее и среднее бакетных средних 
round(np.mean(data_2["time"]), 5), round(np.mean(bucketed_data_2["mu"]), 5)

In [ ]:
round(np.std(data_2["time"]), 5), round(np.mean(bucketed_data_2["std"]), 5)

In [ ]:
control_bucket = bucketed_data_2[bucketed_data_2.con_treat == 0]
treatment_bucket = bucketed_data_2[bucketed_data_2.con_treat == 1]
continious_result(control_bucket, treatment_bucket, 'mu')

In [ ]:
### Testing converted

In [ ]:
fig = px.histogram(data_2, x="converted",
                   color='con_treat', barmode='group',
                   height=400)
fig.show()

In [ ]:
proportion_result(control, treatment, 'converted')